In [6]:
import pandas as pd
import numpy as np
import math
import os
import re
import string
import nltk
import textstat #(https://pypi.org/project/textstat/)

#nltk.download()

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Check current working directory.
retval = os.getcwd()
os.chdir('/Users/bhumikakhandelwal/Documents/Documents/Documents/Academics/Capstone/ProjectWork/Capstone_NLP_DIWD/SEC_Output_Files/parsed_files')

## Read the csv file with all listings  10K
parsed10Q = pd.read_csv('./parsedText10Q.csv', encoding="ISO-8859-1")

In [8]:
parsed10Q

,CompanyName,TypeOfFiling,Year,Quarter,FSS,RiskFactors,MDA,MarketRisk,LegalProceedings,ControlProcedures,AccessedURL
0,FEDEX CORP,10-Q,2018,1,NaN,Item 1A. Risk Factors Other than the risk fact...,Item 2. Management's Discussion and Analysis o...,Item 3. Quantitative and Qualitative Disclosur...,Item 1. Legal Proceedings For a description of...,Item 4. Controls and Procedures The management...,https://www.sec.gov/Archives/edgar/data/104891...
1,Seagate Technology plc,10-Q,2018,1,ITEM 1. FINANCIAL STATEMENTS Table of Contents...,ITEM 1A. RISK FACTORS There have been no mat...,ITEM 2. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,ITEM 3. QUANTITATIVE AND QUALITATIVE DISCLOSUR...,ITEM 1. LEGAL PROCEEDINGS For a discussion of ...,ITEM 4. CONTROLS AND PROCEDURES An evaluation ...,https://www.sec.gov/Archives/edgar/data/113778...
2,ORACLE CORP,10-Q,2018,1,Item 1. Financial Statements (Unaudited) ORACL...,Item 1A. Risk Factors In addition to the ot...,Item 2. Management's Discussion and Analysis o...,Item 3. Quantitative and Qualitative Disclo...,Item 1. Legal Proceedings The material set ...,Item 4. Controls and Procedures Evaluation ...,https://www.sec.gov/Archives/edgar/data/134143...
3,VARIAN MEDICAL SYSTEMS INC,10-Q,2018,1,Item 1. Unaudited Financial Statements VARIAN ...,Item 1A. Risk Factors There were no material c...,Item 2. Management's Discussion and Analysis o...,Item 3. Quantitative and Qualitative Disclosur...,Item 1. Legal Proceedings We are subject to va...,Item 4. Controls and Procedures (a) Disclosure...,https://www.sec.gov/Archives/edgar/data/203527...
4,FRANKLIN RESOURCES INC,10-Q,2018,1,Item 1. Financial Statements. FRANKLIN RESOURC...,Item 1A. Risk Factors. Our Form 10-K for the f...,Item 2. Management's Discussion and Analysis o...,Item 3. Quantitative and Qualitative Disclosur...,Item 1. Legal Proceedings. For a description o...,Item 4. Controls and Procedures. The Company's...,https://www.sec.gov/Archives/edgar/data/38777/...
5,GENERAL MILLS INC,10-Q,2018,1,Item 1. Financial Statements Consolidated Stat...,Item 1A. Risk Factors. There can be no assuran...,Item 2. Management's Discussion and Analysis o...,Item 3. Quantitative and Qualitative Disclosur...,NaN,"Item 4. Controls and Procedures. We, under the...",https://www.sec.gov/Archives/edgar/data/40704/...
6,ATMOS ENERGY CORP,10-Q,2018,1,Item 1. Financial Statements ATMOS ENERGY CORP...,NaN,Item 2. Management's Discussion and Analysis o...,Item 3. Quantitative and Qualitative Disclosur...,Item 1. Legal Proceedings During the three mon...,Item 4. Controls and Procedures Management's E...,https://www.sec.gov/Archives/edgar/data/731802...
7,MICROSOFT CORP,10-Q,2018,1,ITEM 1. FINANCIAL STATEMENTS INCOME STATEMENTS...,ITEM 1A. RISK FACTORS Our operations and finan...,ITEM 2. MANAGEMENT'S DISCUSSION AND ANALYSIS O...,ITEM 3. QUANTITATIVE AND QUALITATIVE DISCLOSUR...,ITEM 1. LEGAL PROCEEDINGS Refer to Note 16 - ...,ITEM 4. CONTROLS AND PROCEDURES Under the supe...,https://www.sec.gov/Archives/edgar/data/789019...
8,MICROCHIP TECHNOLOGY INC,10-Q,2018,1,NaN,Item 1A. Risk Factors When evaluating Microch...,Item 2. Management's Discussion and Analysi...,Item 3. Quantitative and Qualitative Disclosu...,Item 1. Legal Proceedings Refer to Note 15 to ...,Item 4. Controls and Procedures Evaluation of...,https://www.sec.gov/Archives/edgar/data/827054...
9,STARBUCKS CORP,10-Q,2018,1,Item 1. Financial Statements STARBUCKS CORPORA...,Item 1A. Risk Factors There have been no mater...,Item 2. Management's Discussion and Analysis o...,Item 3. Quantitative and Qualitative Disclosur...,Item 1. Legal Proceedings Starbucks is party t...,Item 4. Controls and Procedures We maintain d...,https://www.sec.gov/Archives/edgar/data/829224...




Description of the items in the SEC Filings 10Q:
1. ITEM1: Business Description
2. ITEM1A: Risk Factors
3. ITEM2: Management Discussion and Analysis of Financial Conditions and Results of Operations
4. ITEM3: Quantitative and Qualitative Disclosures About Market Risk
5. ITEM1: Legal Proceedings
6. ITEM4: Controls and Procedures

In [9]:
# Change the names of the columns 10q 
parsed10Q.columns = ['CompanyName','TypeOfFiling','Year','Quater','Business_Desc','Risk_Factors','Management_Discussion','Market_Risk','Legal_Proceedings','Controls_Proc','AccessedURL']

# Tokenize and stemming

In [10]:
def nltk_noise_removal(text):
    tokens = word_tokenize(text) #split into words
    tokens = [w.upper() for w in tokens]
    
    #remove punctuations
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    
    #remove non-alphabetic words
    words = [word for word in stripped if word.isalpha()]
    
    #filter out stop words  ---- english
    stop_words = stopwords.words('english') ## stop words from english
    stop_words = [words.upper() for words in stop_words]
    
    #filter stop words mcdonald
    mcdonald_file = open('StopWords_Generic.txt','r')
    mcdonald_stopwords = mcdonald_file.readlines()
    sw = [words for words in mcdonald_stopwords]
    a = [s.rstrip() for s in sw] ### stopwords from mcdonald
    
    stopwords_all = stop_words + a
    words = [w for w in words if not w in stopwords_all]
    #print(words)
    return words

# for lines in data:
#     corpus = nltk_noise_removal(lines)

# corpus = nltk_noise_removal(parsed10K['Risk_Factors'][4])
# print(corpus)
    
    ### Optional -- in the resources they have asked to skip this in order to keep the original words
#     porter = PorterStemmer()
#     stemmed = [porter.stem(word) for word in tokens]
#     print(stemmed[:100])



# Sentiment Analysis

In [11]:

import re
 
# Loughran and McDonald Sentiment Word Lists (https://sraf.nd.edu/textual-analysis/resources/)
lmdict = {'Negative': ['abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonments', 'abandons', 'abdicated',
                       'abdicates', 'abdicating', 'abdication', 'abdications', 'aberrant', 'aberration', 'aberrational',
                       'aberrations', 'abetting', 'abnormal', 'abnormalities', 'abnormality', 'abnormally', 'abolish',
                       'abolished', 'abolishes', 'abolishing', 'abrogate', 'abrogated', 'abrogates', 'abrogating',
                       'abrogation', 'abrogations', 'abrupt', 'abruptly', 'abruptness', 'absence', 'absences',
                       'absenteeism', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abusively', 'abusiveness',
                       'accident', 'accidental', 'accidentally', 'accidents', 'accusation', 'accusations', 'accuse',
                       'accused', 'accuses', 'accusing', 'acquiesce', 'acquiesced', 'acquiesces', 'acquiescing',
                       'acquit', 'acquits', 'acquittal', 'acquittals', 'acquitted', 'acquitting', 'adulterate',
                       'adulterated', 'adulterating', 'adulteration', 'adulterations', 'adversarial', 'adversaries',
                       'adversary', 'adverse', 'adversely', 'adversities', 'adversity', 'aftermath', 'aftermaths',
                       'against', 'aggravate', 'aggravated', 'aggravates', 'aggravating', 'aggravation', 'aggravations',
                       'alerted', 'alerting', 'alienate', 'alienated', 'alienates', 'alienating', 'alienation',
                       'alienations', 'allegation', 'allegations', 'allege', 'alleged', 'allegedly', 'alleges',
                       'alleging', 'annoy', 'annoyance', 'annoyances', 'annoyed', 'annoying', 'annoys', 'annul',
                       'annulled', 'annulling', 'annulment', 'annulments', 'annuls', 'anomalies', 'anomalous',
                       'anomalously', 'anomaly', 'anticompetitive', 'antitrust', 'argue', 'argued', 'arguing',
                       'argument', 'argumentative', 'arguments', 'arrearage', 'arrearages', 'arrears', 'arrest',
                       'arrested', 'arrests', 'artificially', 'assault', 'assaulted', 'assaulting', 'assaults',
                       'assertions', 'attrition', 'aversely', 'backdating', 'bad', 'bail', 'bailout', 'balk', 'balked',
                       'bankrupt', 'bankruptcies', 'bankruptcy', 'bankrupted', 'bankrupting', 'bankrupts', 'bans',
                       'barred', 'barrier', 'barriers', 'bottleneck', 'bottlenecks', 'boycott', 'boycotted',
                       'boycotting', 'boycotts', 'breach', 'breached', 'breaches', 'breaching', 'break', 'breakage',
                       'breakages', 'breakdown', 'breakdowns', 'breaking', 'breaks', 'bribe', 'bribed', 'briberies',
                       'bribery', 'bribes', 'bribing', 'bridge', 'broken', 'burden', 'burdened', 'burdening', 'burdens',
                       'burdensome', 'burned', 'calamities', 'calamitous', 'calamity', 'cancel', 'canceled',
                       'canceling', 'cancellation', 'cancellations', 'cancelled', 'cancelling', 'cancels', 'careless',
                       'carelessly', 'carelessness', 'catastrophe', 'catastrophes', 'catastrophic', 'catastrophically',
                       'caution', 'cautionary', 'cautioned', 'cautioning', 'cautions', 'cease', 'ceased', 'ceases',
                       'ceasing', 'censure', 'censured', 'censures', 'censuring', 'challenge', 'challenged',
                       'challenges', 'challenging', 'chargeoffs', 'circumvent', 'circumvented', 'circumventing',
                       'circumvention', 'circumventions', 'circumvents', 'claiming', 'claims', 'clawback', 'closed',
                       'closeout', 'closeouts', 'closing', 'closings', 'closure', 'closures', 'coerce', 'coerced',
                       'coerces', 'coercing', 'coercion', 'coercive', 'collapse', 'collapsed', 'collapses',
                       'collapsing', 'collision', 'collisions', 'collude', 'colluded', 'colludes', 'colluding',
                       'collusion', 'collusions', 'collusive', 'complain', 'complained', 'complaining', 'complains',
                       'complaint', 'complaints', 'complicate', 'complicated', 'complicates', 'complicating',
                       'complication', 'complications', 'compulsion', 'concealed', 'concealing', 'concede', 'conceded',
                       'concedes', 'conceding', 'concern', 'concerned', 'concerns', 'conciliating', 'conciliation',
                       'conciliations', 'condemn', 'condemnation', 'condemnations', 'condemned', 'condemning',
                       'condemns', 'condone', 'condoned', 'confess', 'confessed', 'confesses', 'confessing',
                       'confession', 'confine', 'confined', 'confinement', 'confinements', 'confines', 'confining',
                       'confiscate', 'confiscated', 'confiscates', 'confiscating', 'confiscation', 'confiscations',
                       'conflict', 'conflicted', 'conflicting', 'conflicts', 'confront', 'confrontation',
                       'confrontational', 'confrontations', 'confronted', 'confronting', 'confronts', 'confuse',
                       'confused', 'confuses', 'confusing', 'confusingly', 'confusion', 'conspiracies', 'conspiracy',
                       'conspirator', 'conspiratorial', 'conspirators', 'conspire', 'conspired', 'conspires',
                       'conspiring', 'contempt', 'contend', 'contended', 'contending', 'contends', 'contention',
                       'contentions', 'contentious', 'contentiously', 'contested', 'contesting', 'contraction',
                       'contractions', 'contradict', 'contradicted', 'contradicting', 'contradiction', 'contradictions',
                       'contradictory', 'contradicts', 'contrary', 'controversial', 'controversies', 'controversy',
                       'convict', 'convicted', 'convicting', 'conviction', 'convictions', 'corrected', 'correcting',
                       'correction', 'corrections', 'corrects', 'corrupt', 'corrupted', 'corrupting', 'corruption',
                       'corruptions', 'corruptly', 'corruptness', 'costly', 'counterclaim', 'counterclaimed',
                       'counterclaiming', 'counterclaims', 'counterfeit', 'counterfeited', 'counterfeiter',
                       'counterfeiters', 'counterfeiting', 'counterfeits', 'countermeasure', 'countermeasures', 'crime',
                       'crimes', 'criminal', 'criminally', 'criminals', 'crises', 'crisis', 'critical', 'critically',
                       'criticism', 'criticisms', 'criticize', 'criticized', 'criticizes', 'criticizing', 'crucial',
                       'crucially', 'culpability', 'culpable', 'culpably', 'cumbersome', 'curtail', 'curtailed',
                       'curtailing', 'curtailment', 'curtailments', 'curtails', 'cut', 'cutback', 'cutbacks',
                       'cyberattack', 'cyberattacks', 'cyberbullying', 'cybercrime', 'cybercrimes', 'cybercriminal',
                       'cybercriminals', 'damage', 'damaged', 'damages', 'damaging', 'dampen', 'dampened', 'danger',
                       'dangerous', 'dangerously', 'dangers', 'deadlock', 'deadlocked', 'deadlocking', 'deadlocks',
                       'deadweight', 'deadweights', 'debarment', 'debarments', 'debarred', 'deceased', 'deceit',
                       'deceitful', 'deceitfulness', 'deceive', 'deceived', 'deceives', 'deceiving', 'deception',
                       'deceptions', 'deceptive', 'deceptively', 'decline', 'declined', 'declines', 'declining',
                       'deface', 'defaced', 'defacement', 'defamation', 'defamations', 'defamatory', 'defame',
                       'defamed', 'defames', 'defaming', 'default', 'defaulted', 'defaulting', 'defaults', 'defeat',
                       'defeated', 'defeating', 'defeats', 'defect', 'defective', 'defects', 'defend', 'defendant',
                       'defendants', 'defended', 'defending', 'defends', 'defensive', 'defer', 'deficiencies',
                       'deficiency', 'deficient', 'deficit', 'deficits', 'defraud', 'defrauded', 'defrauding',
                       'defrauds', 'defunct', 'degradation', 'degradations', 'degrade', 'degraded', 'degrades',
                       'degrading', 'delay', 'delayed', 'delaying', 'delays', 'deleterious', 'deliberate',
                       'deliberated', 'deliberately', 'delinquencies', 'delinquency', 'delinquent', 'delinquently',
                       'delinquents', 'delist', 'delisted', 'delisting', 'delists', 'demise', 'demised', 'demises',
                       'demising', 'demolish', 'demolished', 'demolishes', 'demolishing', 'demolition', 'demolitions',
                       'demote', 'demoted', 'demotes', 'demoting', 'demotion', 'demotions', 'denial', 'denials',
                       'denied', 'denies', 'denigrate', 'denigrated', 'denigrates', 'denigrating', 'denigration',
                       'deny', 'denying', 'deplete', 'depleted', 'depletes', 'depleting', 'depletion', 'depletions',
                       'deprecation', 'depress', 'depressed', 'depresses', 'depressing', 'deprivation', 'deprive',
                       'deprived', 'deprives', 'depriving', 'derelict', 'dereliction', 'derogatory', 'destabilization',
                       'destabilize', 'destabilized', 'destabilizing', 'destroy', 'destroyed', 'destroying', 'destroys',
                       'destruction', 'destructive', 'detain', 'detained', 'detention', 'detentions', 'deter',
                       'deteriorate', 'deteriorated', 'deteriorates', 'deteriorating', 'deterioration',
                       'deteriorations', 'deterred', 'deterrence', 'deterrences', 'deterrent', 'deterrents',
                       'deterring', 'deters', 'detract', 'detracted', 'detracting', 'detriment', 'detrimental',
                       'detrimentally', 'detriments', 'devalue', 'devalued', 'devalues', 'devaluing', 'devastate',
                       'devastated', 'devastating', 'devastation', 'deviate', 'deviated', 'deviates', 'deviating',
                       'deviation', 'deviations', 'devolve', 'devolved', 'devolves', 'devolving', 'difficult',
                       'difficulties', 'difficultly', 'difficulty', 'diminish', 'diminished', 'diminishes',
                       'diminishing', 'diminution', 'disadvantage', 'disadvantaged', 'disadvantageous', 'disadvantages',
                       'disaffiliation', 'disagree', 'disagreeable', 'disagreed', 'disagreeing', 'disagreement',
                       'disagreements', 'disagrees', 'disallow', 'disallowance', 'disallowances', 'disallowed',
                       'disallowing', 'disallows', 'disappear', 'disappearance', 'disappearances', 'disappeared',
                       'disappearing', 'disappears', 'disappoint', 'disappointed', 'disappointing', 'disappointingly',
                       'disappointment', 'disappointments', 'disappoints', 'disapproval', 'disapprovals', 'disapprove',
                       'disapproved', 'disapproves', 'disapproving', 'disassociates', 'disassociating',
                       'disassociation', 'disassociations', 'disaster', 'disasters', 'disastrous', 'disastrously',
                       'disavow', 'disavowal', 'disavowed', 'disavowing', 'disavows', 'disciplinary', 'disclaim',
                       'disclaimed', 'disclaimer', 'disclaimers', 'disclaiming', 'disclaims', 'disclose', 'disclosed',
                       'discloses', 'disclosing', 'discontinuance', 'discontinuances', 'discontinuation',
                       'discontinuations', 'discontinue', 'discontinued', 'discontinues', 'discontinuing', 'discourage',
                       'discouraged', 'discourages', 'discouraging', 'discredit', 'discredited', 'discrediting',
                       'discredits', 'discrepancies', 'discrepancy', 'disfavor', 'disfavored', 'disfavoring',
                       'disfavors', 'disgorge', 'disgorged', 'disgorgement', 'disgorgements', 'disgorges', 'disgorging',
                       'disgrace', 'disgraceful', 'disgracefully', 'dishonest', 'dishonestly', 'dishonesty', 'dishonor',
                       'dishonorable', 'dishonorably', 'dishonored', 'dishonoring', 'dishonors', 'disincentives',
                       'disinterested', 'disinterestedly', 'disinterestedness', 'disloyal', 'disloyally', 'disloyalty',
                       'dismal', 'dismally', 'dismiss', 'dismissal', 'dismissals', 'dismissed', 'dismisses',
                       'dismissing', 'disorderly', 'disparage', 'disparaged', 'disparagement', 'disparagements',
                       'disparages', 'disparaging', 'disparagingly', 'disparities', 'disparity', 'displace',
                       'displaced', 'displacement', 'displacements', 'displaces', 'displacing', 'dispose', 'dispossess',
                       'dispossessed', 'dispossesses', 'dispossessing', 'disproportion', 'disproportional',
                       'disproportionate', 'disproportionately', 'dispute', 'disputed', 'disputes', 'disputing',
                       'disqualification', 'disqualifications', 'disqualified', 'disqualifies', 'disqualify',
                       'disqualifying', 'disregard', 'disregarded', 'disregarding', 'disregards', 'disreputable',
                       'disrepute', 'disrupt', 'disrupted', 'disrupting', 'disruption', 'disruptions', 'disruptive',
                       'disrupts', 'dissatisfaction', 'dissatisfied', 'dissent', 'dissented', 'dissenter', 'dissenters',
                       'dissenting', 'dissents', 'dissident', 'dissidents', 'dissolution', 'dissolutions', 'distort',
                       'distorted', 'distorting', 'distortion', 'distortions', 'distorts', 'distract', 'distracted',
                       'distracting', 'distraction', 'distractions', 'distracts', 'distress', 'distressed', 'disturb',
                       'disturbance', 'disturbances', 'disturbed', 'disturbing', 'disturbs', 'diversion', 'divert',
                       'diverted', 'diverting', 'diverts', 'divest', 'divested', 'divesting', 'divestiture',
                       'divestitures', 'divestment', 'divestments', 'divests', 'divorce', 'divorced', 'divulge',
                       'divulged', 'divulges', 'divulging', 'doubt', 'doubted', 'doubtful', 'doubts', 'downgrade',
                       'downgraded', 'downgrades', 'downgrading', 'downsize', 'downsized', 'downsizes', 'downsizing',
                       'downsizings', 'downtime', 'downtimes', 'downturn', 'downturns', 'downward', 'downwards', 'drag',
                       'drastic', 'drastically', 'drawback', 'drawbacks', 'dropped', 'drought', 'droughts', 'duress',
                       'dysfunction', 'dysfunctional', 'dysfunctions', 'easing', 'egregious', 'egregiously', 'embargo',
                       'embargoed', 'embargoes', 'embargoing', 'embarrass', 'embarrassed', 'embarrasses',
                       'embarrassing', 'embarrassment', 'embarrassments', 'embezzle', 'embezzled', 'embezzlement',
                       'embezzlements', 'embezzler', 'embezzles', 'embezzling', 'encroach', 'encroached', 'encroaches',
                       'encroaching', 'encroachment', 'encroachments', 'encumber', 'encumbered', 'encumbering',
                       'encumbers', 'encumbrance', 'encumbrances', 'endanger', 'endangered', 'endangering',
                       'endangerment', 'endangers', 'enjoin', 'enjoined', 'enjoining', 'enjoins', 'erode', 'eroded',
                       'erodes', 'eroding', 'erosion', 'erratic', 'erratically', 'erred', 'erring', 'erroneous',
                       'erroneously', 'error', 'errors', 'errs', 'escalate', 'escalated', 'escalates', 'escalating',
                       'evade', 'evaded', 'evades', 'evading', 'evasion', 'evasions', 'evasive', 'evict', 'evicted',
                       'evicting', 'eviction', 'evictions', 'evicts', 'exacerbate', 'exacerbated', 'exacerbates',
                       'exacerbating', 'exacerbation', 'exacerbations', 'exaggerate', 'exaggerated', 'exaggerates',
                       'exaggerating', 'exaggeration', 'excessive', 'excessively', 'exculpate', 'exculpated',
                       'exculpates', 'exculpating', 'exculpation', 'exculpations', 'exculpatory', 'exonerate',
                       'exonerated', 'exonerates', 'exonerating', 'exoneration', 'exonerations', 'exploit',
                       'exploitation', 'exploitations', 'exploitative', 'exploited', 'exploiting', 'exploits', 'expose',
                       'exposed', 'exposes', 'exposing', 'expropriate', 'expropriated', 'expropriates', 'expropriating',
                       'expropriation', 'expropriations', 'expulsion', 'expulsions', 'extenuating', 'fail', 'failed',
                       'failing', 'failings', 'fails', 'failure', 'failures', 'fallout', 'false', 'falsely',
                       'falsification', 'falsifications', 'falsified', 'falsifies', 'falsify', 'falsifying', 'falsity',
                       'fatalities', 'fatality', 'fatally', 'fault', 'faulted', 'faults', 'faulty', 'fear', 'fears',
                       'felonies', 'felonious', 'felony', 'fictitious', 'fined', 'fines', 'fired', 'firing', 'flaw',
                       'flawed', 'flaws', 'forbid', 'forbidden', 'forbidding', 'forbids', 'force', 'forced', 'forcing',
                       'foreclose', 'foreclosed', 'forecloses', 'foreclosing', 'foreclosure', 'foreclosures', 'forego',
                       'foregoes', 'foregone', 'forestall', 'forestalled', 'forestalling', 'forestalls', 'forfeit',
                       'forfeited', 'forfeiting', 'forfeits', 'forfeiture', 'forfeitures', 'forgers', 'forgery',
                       'fraud', 'frauds', 'fraudulence', 'fraudulent', 'fraudulently', 'frivolous', 'frivolously',
                       'frustrate', 'frustrated', 'frustrates', 'frustrating', 'frustratingly', 'frustration',
                       'frustrations', 'fugitive', 'fugitives', 'gratuitous', 'gratuitously', 'grievance', 'grievances',
                       'grossly', 'groundless', 'guilty', 'halt', 'halted', 'hamper', 'hampered', 'hampering',
                       'hampers', 'harass', 'harassed', 'harassing', 'harassment', 'hardship', 'hardships', 'harm',
                       'harmed', 'harmful', 'harmfully', 'harming', 'harms', 'harsh', 'harsher', 'harshest', 'harshly',
                       'harshness', 'hazard', 'hazardous', 'hazards', 'hinder', 'hindered', 'hindering', 'hinders',
                       'hindrance', 'hindrances', 'hostile', 'hostility', 'hurt', 'hurting', 'idle', 'idled', 'idling',
                       'ignore', 'ignored', 'ignores', 'ignoring', 'ill', 'illegal', 'illegalities', 'illegality',
                       'illegally', 'illegible', 'illicit', 'illicitly', 'illiquid', 'illiquidity', 'imbalance',
                       'imbalances', 'immature', 'immoral', 'impair', 'impaired', 'impairing', 'impairment',
                       'impairments', 'impairs', 'impasse', 'impasses', 'impede', 'impeded', 'impedes', 'impediment',
                       'impediments', 'impeding', 'impending', 'imperative', 'imperfection', 'imperfections', 'imperil',
                       'impermissible', 'implicate', 'implicated', 'implicates', 'implicating', 'impossibility',
                       'impossible', 'impound', 'impounded', 'impounding', 'impounds', 'impracticable', 'impractical',
                       'impracticalities', 'impracticality', 'imprisonment', 'improper', 'improperly', 'improprieties',
                       'impropriety', 'imprudent', 'imprudently', 'inability', 'inaccessible', 'inaccuracies',
                       'inaccuracy', 'inaccurate', 'inaccurately', 'inaction', 'inactions', 'inactivate', 'inactivated',
                       'inactivates', 'inactivating', 'inactivation', 'inactivations', 'inactivity', 'inadequacies',
                       'inadequacy', 'inadequate', 'inadequately', 'inadvertent', 'inadvertently', 'inadvisability',
                       'inadvisable', 'inappropriate', 'inappropriately', 'inattention', 'incapable', 'incapacitated',
                       'incapacity', 'incarcerate', 'incarcerated', 'incarcerates', 'incarcerating', 'incarceration',
                       'incarcerations', 'incidence', 'incidences', 'incident', 'incidents', 'incompatibilities',
                       'incompatibility', 'incompatible', 'incompetence', 'incompetency', 'incompetent',
                       'incompetently', 'incompetents', 'incomplete', 'incompletely', 'incompleteness', 'inconclusive',
                       'inconsistencies', 'inconsistency', 'inconsistent', 'inconsistently', 'inconvenience',
                       'inconveniences', 'inconvenient', 'incorrect', 'incorrectly', 'incorrectness', 'indecency',
                       'indecent', 'indefeasible', 'indefeasibly', 'indict', 'indictable', 'indicted', 'indicting',
                       'indictment', 'indictments', 'ineffective', 'ineffectively', 'ineffectiveness', 'inefficiencies',
                       'inefficiency', 'inefficient', 'inefficiently', 'ineligibility', 'ineligible', 'inequitable',
                       'inequitably', 'inequities', 'inequity', 'inevitable', 'inexperience', 'inexperienced',
                       'inferior', 'inflicted', 'infraction', 'infractions', 'infringe', 'infringed', 'infringement',
                       'infringements', 'infringes', 'infringing', 'inhibited', 'inimical', 'injunction', 'injunctions',
                       'injure', 'injured', 'injures', 'injuries', 'injuring', 'injurious', 'injury', 'inordinate',
                       'inordinately', 'inquiry', 'insecure', 'insensitive', 'insolvencies', 'insolvency', 'insolvent',
                       'instability', 'insubordination', 'insufficiency', 'insufficient', 'insufficiently',
                       'insurrection', 'insurrections', 'intentional', 'interfere', 'interfered', 'interference',
                       'interferences', 'interferes', 'interfering', 'intermittent', 'intermittently', 'interrupt',
                       'interrupted', 'interrupting', 'interruption', 'interruptions', 'interrupts', 'intimidation',
                       'intrusion', 'invalid', 'invalidate', 'invalidated', 'invalidates', 'invalidating',
                       'invalidation', 'invalidity', 'investigate', 'investigated', 'investigates', 'investigating',
                       'investigation', 'investigations', 'involuntarily', 'involuntary', 'irreconcilable',
                       'irreconcilably', 'irrecoverable', 'irrecoverably', 'irregular', 'irregularities',
                       'irregularity', 'irregularly', 'irreparable', 'irreparably', 'irreversible', 'jeopardize',
                       'jeopardized', 'justifiable', 'kickback', 'kickbacks', 'knowingly', 'lack', 'lacked', 'lacking',
                       'lackluster', 'lacks', 'lag', 'lagged', 'lagging', 'lags', 'lapse', 'lapsed', 'lapses',
                       'lapsing', 'late', 'laundering', 'layoff', 'layoffs', 'lie', 'limitation', 'limitations',
                       'lingering', 'liquidate', 'liquidated', 'liquidates', 'liquidating', 'liquidation',
                       'liquidations', 'liquidator', 'liquidators', 'litigant', 'litigants', 'litigate', 'litigated',
                       'litigates', 'litigating', 'litigation', 'litigations', 'lockout', 'lockouts', 'lose', 'loses',
                       'losing', 'loss', 'losses', 'lost', 'lying', 'malfeasance', 'malfunction', 'malfunctioned',
                       'malfunctioning', 'malfunctions', 'malice', 'malicious', 'maliciously', 'malpractice',
                       'manipulate', 'manipulated', 'manipulates', 'manipulating', 'manipulation', 'manipulations',
                       'manipulative', 'markdown', 'markdowns', 'misapplication', 'misapplications', 'misapplied',
                       'misapplies', 'misapply', 'misapplying', 'misappropriate', 'misappropriated', 'misappropriates',
                       'misappropriating', 'misappropriation', 'misappropriations', 'misbranded', 'miscalculate',
                       'miscalculated', 'miscalculates', 'miscalculating', 'miscalculation', 'miscalculations',
                       'mischaracterization', 'mischief', 'misclassification', 'misclassifications', 'misclassified',
                       'misclassify', 'miscommunication', 'misconduct', 'misdated', 'misdemeanor', 'misdemeanors',
                       'misdirected', 'mishandle', 'mishandled', 'mishandles', 'mishandling', 'misinform',
                       'misinformation', 'misinformed', 'misinforming', 'misinforms', 'misinterpret',
                       'misinterpretation', 'misinterpretations', 'misinterpreted', 'misinterpreting', 'misinterprets',
                       'misjudge', 'misjudged', 'misjudges', 'misjudging', 'misjudgment', 'misjudgments', 'mislabel',
                       'mislabeled', 'mislabeling', 'mislabelled', 'mislabels', 'mislead', 'misleading', 'misleadingly',
                       'misleads', 'misled', 'mismanage', 'mismanaged', 'mismanagement', 'mismanages', 'mismanaging',
                       'mismatch', 'mismatched', 'mismatches', 'mismatching', 'misplaced', 'misprice', 'mispricing',
                       'mispricings', 'misrepresent', 'misrepresentation', 'misrepresentations', 'misrepresented',
                       'misrepresenting', 'misrepresents', 'miss', 'missed', 'misses', 'misstate', 'misstated',
                       'misstatement', 'misstatements', 'misstates', 'misstating', 'misstep', 'missteps', 'mistake',
                       'mistaken', 'mistakenly', 'mistakes', 'mistaking', 'mistrial', 'mistrials', 'misunderstand',
                       'misunderstanding', 'misunderstandings', 'misunderstood', 'misuse', 'misused', 'misuses',
                       'misusing', 'monopolistic', 'monopolists', 'monopolization', 'monopolize', 'monopolized',
                       'monopolizes', 'monopolizing', 'monopoly', 'moratoria', 'moratorium', 'moratoriums',
                       'mothballed', 'mothballing', 'negative', 'negatively', 'negatives', 'neglect', 'neglected',
                       'neglectful', 'neglecting', 'neglects', 'negligence', 'negligences', 'negligent', 'negligently',
                       'nonattainment', 'noncompetitive', 'noncompliance', 'noncompliances', 'noncompliant',
                       'noncomplying', 'nonconforming', 'nonconformities', 'nonconformity', 'nondisclosure',
                       'nonfunctional', 'nonpayment', 'nonpayments', 'nonperformance', 'nonperformances',
                       'nonperforming', 'nonproducing', 'nonproductive', 'nonrecoverable', 'nonrenewal', 'nuisance',
                       'nuisances', 'nullification', 'nullifications', 'nullified', 'nullifies', 'nullify',
                       'nullifying', 'objected', 'objecting', 'objection', 'objectionable', 'objectionably',
                       'objections', 'obscene', 'obscenity', 'obsolescence', 'obsolete', 'obstacle', 'obstacles',
                       'obstruct', 'obstructed', 'obstructing', 'obstruction', 'obstructions', 'offence', 'offences',
                       'offend', 'offended', 'offender', 'offenders', 'offending', 'offends', 'omission', 'omissions',
                       'omit', 'omits', 'omitted', 'omitting', 'onerous', 'opportunistic', 'opportunistically',
                       'oppose', 'opposed', 'opposes', 'opposing', 'opposition', 'oppositions', 'outage', 'outages',
                       'outdated', 'outmoded', 'overage', 'overages', 'overbuild', 'overbuilding', 'overbuilds',
                       'overbuilt', 'overburden', 'overburdened', 'overburdening', 'overcapacities', 'overcapacity',
                       'overcharge', 'overcharged', 'overcharges', 'overcharging', 'overcome', 'overcomes',
                       'overcoming', 'overdue', 'overestimate', 'overestimated', 'overestimates', 'overestimating',
                       'overestimation', 'overestimations', 'overload', 'overloaded', 'overloading', 'overloads',
                       'overlook', 'overlooked', 'overlooking', 'overlooks', 'overpaid', 'overpayment', 'overpayments',
                       'overproduced', 'overproduces', 'overproducing', 'overproduction', 'overrun', 'overrunning',
                       'overruns', 'overshadow', 'overshadowed', 'overshadowing', 'overshadows', 'overstate',
                       'overstated', 'overstatement', 'overstatements', 'overstates', 'overstating', 'oversupplied',
                       'oversupplies', 'oversupply', 'oversupplying', 'overtly', 'overturn', 'overturned',
                       'overturning', 'overturns', 'overvalue', 'overvalued', 'overvaluing', 'panic', 'panics',
                       'penalize', 'penalized', 'penalizes', 'penalizing', 'penalties', 'penalty', 'peril', 'perils',
                       'perjury', 'perpetrate', 'perpetrated', 'perpetrates', 'perpetrating', 'perpetration', 'persist',
                       'persisted', 'persistence', 'persistent', 'persistently', 'persisting', 'persists', 'pervasive',
                       'pervasively', 'pervasiveness', 'petty', 'picket', 'picketed', 'picketing', 'plaintiff',
                       'plaintiffs', 'plea', 'plead', 'pleaded', 'pleading', 'pleadings', 'pleads', 'pleas', 'pled',
                       'poor', 'poorly', 'poses', 'posing', 'postpone', 'postponed', 'postponement', 'postponements',
                       'postpones', 'postponing', 'precipitated', 'precipitous', 'precipitously', 'preclude',
                       'precluded', 'precludes', 'precluding', 'predatory', 'prejudice', 'prejudiced', 'prejudices',
                       'prejudicial', 'prejudicing', 'premature', 'prematurely', 'pressing', 'pretrial', 'preventing',
                       'prevention', 'prevents', 'problem', 'problematic', 'problematical', 'problems', 'prolong',
                       'prolongation', 'prolongations', 'prolonged', 'prolonging', 'prolongs', 'prone', 'prosecute',
                       'prosecuted', 'prosecutes', 'prosecuting', 'prosecution', 'prosecutions', 'protest', 'protested',
                       'protester', 'protesters', 'protesting', 'protestor', 'protestors', 'protests', 'protracted',
                       'protraction', 'provoke', 'provoked', 'provokes', 'provoking', 'punished', 'punishes',
                       'punishing', 'punishment', 'punishments', 'punitive', 'purport', 'purported', 'purportedly',
                       'purporting', 'purports', 'question', 'questionable', 'questionably', 'questioned',
                       'questioning', 'questions', 'quit', 'quitting', 'racketeer', 'racketeering', 'rationalization',
                       'rationalizations', 'rationalize', 'rationalized', 'rationalizes', 'rationalizing',
                       'reassessment', 'reassessments', 'reassign', 'reassigned', 'reassigning', 'reassignment',
                       'reassignments', 'reassigns', 'recall', 'recalled', 'recalling', 'recalls', 'recession',
                       'recessionary', 'recessions', 'reckless', 'recklessly', 'recklessness', 'redact', 'redacted',
                       'redacting', 'redaction', 'redactions', 'redefault', 'redefaulted', 'redefaults', 'redress',
                       'redressed', 'redresses', 'redressing', 'refusal', 'refusals', 'refuse', 'refused', 'refuses',
                       'refusing', 'reject', 'rejected', 'rejecting', 'rejection', 'rejections', 'rejects',
                       'relinquish', 'relinquished', 'relinquishes', 'relinquishing', 'relinquishment',
                       'relinquishments', 'reluctance', 'reluctant', 'renegotiate', 'renegotiated', 'renegotiates',
                       'renegotiating', 'renegotiation', 'renegotiations', 'renounce', 'renounced', 'renouncement',
                       'renouncements', 'renounces', 'renouncing', 'reparation', 'reparations', 'repossessed',
                       'repossesses', 'repossessing', 'repossession', 'repossessions', 'repudiate', 'repudiated',
                       'repudiates', 'repudiating', 'repudiation', 'repudiations', 'resign', 'resignation',
                       'resignations', 'resigned', 'resigning', 'resigns', 'restate', 'restated', 'restatement',
                       'restatements', 'restates', 'restating', 'restructure', 'restructured', 'restructures',
                       'restructuring', 'restructurings', 'retaliate', 'retaliated', 'retaliates', 'retaliating',
                       'retaliation', 'retaliations', 'retaliatory', 'retribution', 'retributions', 'revocation',
                       'revocations', 'revoke', 'revoked', 'revokes', 'revoking', 'ridicule', 'ridiculed', 'ridicules',
                       'ridiculing', 'riskier', 'riskiest', 'risky', 'sabotage', 'sacrifice', 'sacrificed',
                       'sacrifices', 'sacrificial', 'sacrificing', 'scandalous', 'scandals', 'scrutinize',
                       'scrutinized', 'scrutinizes', 'scrutinizing', 'scrutiny', 'secrecy', 'seize', 'seized', 'seizes',
                       'seizing', 'sentenced', 'sentencing', 'serious', 'seriously', 'seriousness', 'setback',
                       'setbacks', 'sever', 'severe', 'severed', 'severely', 'severities', 'severity', 'sharply',
                       'shocked', 'shortage', 'shortages', 'shortfall', 'shortfalls', 'shrinkage', 'shrinkages', 'shut',
                       'shutdown', 'shutdowns', 'shuts', 'shutting', 'slander', 'slandered', 'slanderous', 'slanders',
                       'slippage', 'slippages', 'slow', 'slowdown', 'slowdowns', 'slowed', 'slower', 'slowest',
                       'slowing', 'slowly', 'slowness', 'sluggish', 'sluggishly', 'sluggishness', 'solvencies',
                       'solvency', 'spam', 'spammers', 'spamming', 'staggering', 'stagnant', 'stagnate', 'stagnated',
                       'stagnates', 'stagnating', 'stagnation', 'standstill', 'standstills', 'stolen', 'stoppage',
                       'stoppages', 'stopped', 'stopping', 'stops', 'strain', 'strained', 'straining', 'strains',
                       'stress', 'stressed', 'stresses', 'stressful', 'stressing', 'stringent', 'subjected',
                       'subjecting', 'subjection', 'subpoena', 'subpoenaed', 'subpoenas', 'substandard', 'sue', 'sued',
                       'sues', 'suffer', 'suffered', 'suffering', 'suffers', 'suing', 'summoned', 'summoning',
                       'summons', 'summonses', 'susceptibility', 'susceptible', 'suspect', 'suspected', 'suspects',
                       'suspend', 'suspended', 'suspending', 'suspends', 'suspension', 'suspensions', 'suspicion',
                       'suspicions', 'suspicious', 'suspiciously', 'taint', 'tainted', 'tainting', 'taints', 'tampered',
                       'tense', 'terminate', 'terminated', 'terminates', 'terminating', 'termination', 'terminations',
                       'testify', 'testifying', 'threat', 'threaten', 'threatened', 'threatening', 'threatens',
                       'threats', 'tightening', 'tolerate', 'tolerated', 'tolerates', 'tolerating', 'toleration',
                       'tortuous', 'tortuously', 'tragedies', 'tragedy', 'tragic', 'tragically', 'traumatic', 'trouble',
                       'troubled', 'troubles', 'turbulence', 'turmoil', 'unable', 'unacceptable', 'unacceptably',
                       'unaccounted', 'unannounced', 'unanticipated', 'unapproved', 'unattractive', 'unauthorized',
                       'unavailability', 'unavailable', 'unavoidable', 'unavoidably', 'unaware', 'uncollectable',
                       'uncollected', 'uncollectibility', 'uncollectible', 'uncollectibles', 'uncompetitive',
                       'uncompleted', 'unconscionable', 'unconscionably', 'uncontrollable', 'uncontrollably',
                       'uncontrolled', 'uncorrected', 'uncover', 'uncovered', 'uncovering', 'uncovers', 'undeliverable',
                       'undelivered', 'undercapitalized', 'undercut', 'undercuts', 'undercutting', 'underestimate',
                       'underestimated', 'underestimates', 'underestimating', 'underestimation', 'underfunded',
                       'underinsured', 'undermine', 'undermined', 'undermines', 'undermining', 'underpaid',
                       'underpayment', 'underpayments', 'underpays', 'underperform', 'underperformance',
                       'underperformed', 'underperforming', 'underperforms', 'underproduced', 'underproduction',
                       'underreporting', 'understate', 'understated', 'understatement', 'understatements',
                       'understates', 'understating', 'underutilization', 'underutilized', 'undesirable', 'undesired',
                       'undetected', 'undetermined', 'undisclosed', 'undocumented', 'undue', 'unduly', 'uneconomic',
                       'uneconomical', 'uneconomically', 'unemployed', 'unemployment', 'unethical', 'unethically',
                       'unexcused', 'unexpected', 'unexpectedly', 'unfair', 'unfairly', 'unfavorability', 'unfavorable',
                       'unfavorably', 'unfavourable', 'unfeasible', 'unfit', 'unfitness', 'unforeseeable', 'unforeseen',
                       'unforseen', 'unfortunate', 'unfortunately', 'unfounded', 'unfriendly', 'unfulfilled',
                       'unfunded', 'uninsured', 'unintended', 'unintentional', 'unintentionally', 'unjust',
                       'unjustifiable', 'unjustifiably', 'unjustified', 'unjustly', 'unknowing', 'unknowingly',
                       'unlawful', 'unlawfully', 'unlicensed', 'unliquidated', 'unmarketable', 'unmerchantable',
                       'unmeritorious', 'unnecessarily', 'unnecessary', 'unneeded', 'unobtainable', 'unoccupied',
                       'unpaid', 'unperformed', 'unplanned', 'unpopular', 'unpredictability', 'unpredictable',
                       'unpredictably', 'unpredicted', 'unproductive', 'unprofitability', 'unprofitable', 'unqualified',
                       'unrealistic', 'unreasonable', 'unreasonableness', 'unreasonably', 'unreceptive',
                       'unrecoverable', 'unrecovered', 'unreimbursed', 'unreliable', 'unremedied', 'unreported',
                       'unresolved', 'unrest', 'unsafe', 'unsalable', 'unsaleable', 'unsatisfactory', 'unsatisfied',
                       'unsavory', 'unscheduled', 'unsellable', 'unsold', 'unsound', 'unstabilized', 'unstable',
                       'unsubstantiated', 'unsuccessful', 'unsuccessfully', 'unsuitability', 'unsuitable', 'unsuitably',
                       'unsuited', 'unsure', 'unsuspected', 'unsuspecting', 'unsustainable', 'untenable', 'untimely',
                       'untrusted', 'untruth', 'untruthful', 'untruthfully', 'untruthfulness', 'untruths', 'unusable',
                       'unwanted', 'unwarranted', 'unwelcome', 'unwilling', 'unwillingness', 'upset', 'urgency',
                       'urgent', 'usurious', 'usurp', 'usurped', 'usurping', 'usurps', 'usury', 'vandalism', 'verdict',
                       'verdicts', 'vetoed', 'victims', 'violate', 'violated', 'violates', 'violating', 'violation',
                       'violations', 'violative', 'violator', 'violators', 'violence', 'violent', 'violently',
                       'vitiate', 'vitiated', 'vitiates', 'vitiating', 'vitiation', 'voided', 'voiding', 'volatile',
                       'volatility', 'vulnerabilities', 'vulnerability', 'vulnerable', 'vulnerably', 'warn', 'warned',
                       'warning', 'warnings', 'warns', 'wasted', 'wasteful', 'wasting', 'weak', 'weaken', 'weakened',
                       'weakening', 'weakens', 'weaker', 'weakest', 'weakly', 'weakness', 'weaknesses', 'willfully',
                       'worries', 'worry', 'worrying', 'worse', 'worsen', 'worsened', 'worsening', 'worsens', 'worst',
                       'worthless', 'writedown', 'writedowns', 'writeoff', 'writeoffs', 'wrong', 'wrongdoing',
                       'wrongdoings', 'wrongful', 'wrongfully', 'wrongly'],
          'Positive': ['able', 'abundance', 'abundant', 'acclaimed', 'accomplish', 'accomplished', 'accomplishes',
                       'accomplishing', 'accomplishment', 'accomplishments', 'achieve', 'achieved', 'achievement',
                       'achievements', 'achieves', 'achieving', 'adequately', 'advancement', 'advancements', 'advances',
                       'advancing', 'advantage', 'advantaged', 'advantageous', 'advantageously', 'advantages',
                       'alliance', 'alliances', 'assure', 'assured', 'assures', 'assuring', 'attain', 'attained',
                       'attaining', 'attainment', 'attainments', 'attains', 'attractive', 'attractiveness', 'beautiful',
                       'beautifully', 'beneficial', 'beneficially', 'benefit', 'benefited', 'benefiting', 'benefitted',
                       'benefitting', 'best', 'better', 'bolstered', 'bolstering', 'bolsters', 'boom', 'booming',
                       'boost', 'boosted', 'breakthrough', 'breakthroughs', 'brilliant', 'charitable', 'collaborate',
                       'collaborated', 'collaborates', 'collaborating', 'collaboration', 'collaborations',
                       'collaborative', 'collaborator', 'collaborators', 'compliment', 'complimentary', 'complimented',
                       'complimenting', 'compliments', 'conclusive', 'conclusively', 'conducive', 'confident',
                       'constructive', 'constructively', 'courteous', 'creative', 'creatively', 'creativeness',
                       'creativity', 'delight', 'delighted', 'delightful', 'delightfully', 'delighting', 'delights',
                       'dependability', 'dependable', 'desirable', 'desired', 'despite', 'destined', 'diligent',
                       'diligently', 'distinction', 'distinctions', 'distinctive', 'distinctively', 'distinctiveness',
                       'dream', 'easier', 'easily', 'easy', 'effective', 'efficiencies', 'efficiency', 'efficient',
                       'efficiently', 'empower', 'empowered', 'empowering', 'empowers', 'enable', 'enabled', 'enables',
                       'enabling', 'encouraged', 'encouragement', 'encourages', 'encouraging', 'enhance', 'enhanced',
                       'enhancement', 'enhancements', 'enhances', 'enhancing', 'enjoy', 'enjoyable', 'enjoyably',
                       'enjoyed', 'enjoying', 'enjoyment', 'enjoys', 'enthusiasm', 'enthusiastic', 'enthusiastically',
                       'excellence', 'excellent', 'excelling', 'excels', 'exceptional', 'exceptionally', 'excited',
                       'excitement', 'exciting', 'exclusive', 'exclusively', 'exclusiveness', 'exclusives',
                       'exclusivity', 'exemplary', 'fantastic', 'favorable', 'favorably', 'favored', 'favoring',
                       'favorite', 'favorites', 'friendly', 'gain', 'gained', 'gaining', 'gains', 'good', 'great',
                       'greater', 'greatest', 'greatly', 'greatness', 'happiest', 'happily', 'happiness', 'happy',
                       'highest', 'honor', 'honorable', 'honored', 'honoring', 'honors', 'ideal', 'impress',
                       'impressed', 'impresses', 'impressing', 'impressive', 'impressively', 'improve', 'improved',
                       'improvement', 'improvements', 'improves', 'improving', 'incredible', 'incredibly',
                       'influential', 'informative', 'ingenuity', 'innovate', 'innovated', 'innovates', 'innovating',
                       'innovation', 'innovations', 'innovative', 'innovativeness', 'innovator', 'innovators',
                       'insightful', 'inspiration', 'inspirational', 'integrity', 'invent', 'invented', 'inventing',
                       'invention', 'inventions', 'inventive', 'inventiveness', 'inventor', 'inventors', 'leadership',
                       'leading', 'loyal', 'lucrative', 'meritorious', 'opportunities', 'opportunity', 'optimistic',
                       'outperform', 'outperformed', 'outperforming', 'outperforms', 'perfect', 'perfected',
                       'perfectly', 'perfects', 'pleasant', 'pleasantly', 'pleased', 'pleasure', 'plentiful', 'popular',
                       'popularity', 'positive', 'positively', 'preeminence', 'preeminent', 'premier', 'premiere',
                       'prestige', 'prestigious', 'proactive', 'proactively', 'proficiency', 'proficient',
                       'proficiently', 'profitability', 'profitable', 'profitably', 'progress', 'progressed',
                       'progresses', 'progressing', 'prospered', 'prospering', 'prosperity', 'prosperous', 'prospers',
                       'rebound', 'rebounded', 'rebounding', 'receptive', 'regain', 'regained', 'regaining', 'resolve',
                       'revolutionize', 'revolutionized', 'revolutionizes', 'revolutionizing', 'reward', 'rewarded',
                       'rewarding', 'rewards', 'satisfaction', 'satisfactorily', 'satisfactory', 'satisfied',
                       'satisfies', 'satisfy', 'satisfying', 'smooth', 'smoothing', 'smoothly', 'smooths', 'solves',
                       'solving', 'spectacular', 'spectacularly', 'stability', 'stabilization', 'stabilizations',
                       'stabilize', 'stabilized', 'stabilizes', 'stabilizing', 'stable', 'strength', 'strengthen',
                       'strengthened', 'strengthening', 'strengthens', 'strengths', 'strong', 'stronger', 'strongest',
                       'succeed', 'succeeded', 'succeeding', 'succeeds', 'success', 'successes', 'successful',
                       'successfully', 'superior', 'surpass', 'surpassed', 'surpasses', 'surpassing', 'transparency',
                       'tremendous', 'tremendously', 'unmatched', 'unparalleled', 'unsurpassed', 'upturn', 'upturns',
                       'valuable', 'versatile', 'versatility', 'vibrancy', 'vibrant', 'win', 'winner', 'winners',
                       'winning', 'worthy']}

In [12]:

negate = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt", "ain't", "aren't", "can't",
          "couldn't", "daren't", "didn't", "doesn't", "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt",
          "neither", "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't", "neednt", "needn't",
          "never", "none", "nope", "nor", "not", "nothing", "nowhere", "oughtnt", "shant", "shouldnt", "wasnt",
          "werent", "oughtn't", "shan't", "shouldn't", "wasn't", "weren't", "without", "wont", "wouldnt", "won't",
          "wouldn't", "rarely", "seldom", "despite", "no", "nobody"]

In [13]:
# lmdict['Negative']

def negated(word):
    """
    Determine if preceding word is a negation word
    """
    if word.lower() in negate:
        return True
    else:
        return False



In [14]:

def tone_count_with_negation_check(dict, article):
    """
    Count positive and negative words with negation check. Account for simple negation only for positive words.
    Simple negation is taken to be observations of one of negate words occurring within three words
    preceding a positive words.
    """
    
    pos_count = 0
    neg_count = 0
 
    pos_words = []
    neg_words = []
 
    input_words = re.findall(r'\b([a-zA-Z]+n\'t|[a-zA-Z]+\'s|[a-zA-Z]+)\b', article.lower())
 
    word_count = len(input_words)
 
    for i in range(0, word_count):
        if input_words[i] in dict['Negative']:
            neg_count += 1
            neg_words.append(input_words[i])
        if input_words[i] in dict['Positive']:
            if i >= 3:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]) or negated(input_words[i - 3]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 2:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 1:
                if negated(input_words[i - 1]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 0:
                pos_count += 1
                pos_words.append(input_words[i])
 
    #print('The results with negation check:', end='\n\n')
    #print('The # of positive words:', pos_count)
    #print('The # of negative words:', neg_count)
    #print('The list of found positive words:', pos_words)
    #print('The list of found negative words:', neg_words)
    #print('\n', end='')
 
    results = [word_count, pos_count, neg_count, pos_words, neg_words]

    return results

In [15]:
def remove_duplicate(list_items):
    original = list_items
    b = set()
    unique = []
    
    for x in original:
        if x not in b:
            unique.append(x)
            b.add(x)
    return unique

In [16]:
## OLD Function
## Function to check the sentiment of each items in every company - Item wise

# def sentiments_each_item(dictionary,df,company_name):
#     """
#     Input: dictionary name, Parsed10k dataframe, Company Name for which the results are required
#     Ouput: 
#     Function outputs the dataframe with the sentiments of every items individually of each company
#     """
#     items = ['Business_Desc','Risk_Factors','Management_Discussion','Market_Risk','Financial_Statements','Controls_Proc']
#     company = df['CompanyName'] == company_name
#     req_df = df[company]
#     #print(req_df.Year)
#     req_df = req_df.reset_index(drop=True)
#     temp_df = pd.DataFrame(columns= ['Company','Year','Quater','Section','Posiitvewords','Negativewords','List_Positivevewords','List_Negativevewords'],index=range(5))
#     output = []
    
#     for each in items:
#         output.append(tone_count_with_negation_check(dictionary,req_df[each][0]))
    
    
#     for elements in range(len(output)):
#         year = (list(req_df['Year']))
#         quater = (list(req_df['Quater']))
#         for i in range(5):
       
#             temp_df.loc[i].Company = company_name
#             temp_df.loc[i].Year = year[0]
#             temp_df.loc[i].Quater = quater[0]
#             temp_df.loc[i].Section = items[i]
#             temp_df.loc[i].Posiitvewords = output[i][1]
#             temp_df.loc[i].Negativewords = output[i][2]
#             temp_df.loc[i].List_Positivevewords = ",".join(map(str, remove_duplicate(output[i][3])))   ## remove the repeated words in the list of positive and negative
#             temp_df.loc[i].List_Negativevewords = ",".join(map(str, remove_duplicate(output[i][4])))
  
#     return temp_df
        
        

In [18]:

  ##### NEW function 
import numpy as np
def sentiments_each_item(dictionary,df,company_name):
    """
    Input: dictionary name, parsed1Q dataframe, Company Name for which the results are required
    Ouput: 
    Function outputs the dataframe with the sentiments of every items individually of each company
    Function also outputs the gunning fog index of each section and their total
    """
    items = ['Business_Desc','Risk_Factors','Management_Discussion','Market_Risk','Financial_Statements','Controls_Proc']
    company = df['CompanyName'] == company_name
    req_df = df[company]
    #print(req_df.Year)
    req_df = req_df.reset_index(drop=True)
    temp_df = pd.DataFrame(columns= ['Company','Year','Quater',
                                     'Business_Desc_Posiitvewords','Business_Desc_Negativewords','Business_Desc_List_Positivevewords','Business_Desc_List_Negativevewords','Business_Desc_Gunning_Fog',
                                    'Risk_Factors_Posiitvewords','Risk_Factors_Negativewords','Risk_Factors_List_Positivevewords','Risk_Factors_List_Negativevewords','Risk_Factors_Gunning_Fog',
                                    'Management_Discussion_Posiitvewords','Management_Discussion_Negativewords','Management_Discussion_List_Positivevewords','Management_Discussion_List_Negativevewords','Management_Discussion_Gunning_Fog',
                                    'Market_Risk_Posiitvewords','Market_Risk_Negativewords','Market_Risk_List_Positivevewords','Market_Risk_List_Negativevewords','Market_Risk_Gunning_Fog',
                                    'Financial_Statements_Posiitvewords','Financial_Statements_Negativewords','Financial_Statements_List_Positivevewords','Financial_Statements_List_Negativevewords','Financial_Statements_Gunning_Fog',
                                    'Controls_Proc_Posiitvewords','Controls_Proc_Negativewords','Controls_Proc_List_Positivevewords','Controls_Proc_List_Negativevewords','Controls_Proc_Gunning_Fog',
                                    'Total_Gunning_Fog'])

    
    
    
    output = []
    gunning_fog = []
    
    req_df = pd.DataFrame(req_df.replace(np.nan, 'no text', regex=True))
    
    ####### Business Desc
    Business_Desc_Posiitvewords = []
    Business_Desc_Negativewords = []
    Business_Desc_List_Positivevewords = []
    Business_Desc_List_Negativevewords = []
    Business_Desc_Gunning_Fog = []
   
    
    for row in req_df['Business_Desc']:
        Business_Desc_Posiitvewords.append(tone_count_with_negation_check(dictionary,row)[1])
        Business_Desc_Negativewords.append(tone_count_with_negation_check(dictionary,row)[2])
        Business_Desc_List_Positivevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[3]))))
        Business_Desc_List_Negativevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[4]))))
        Business_Desc_Gunning_Fog.append(textstat.gunning_fog(row))
       
    req_df['Business_Desc_Posiitvewords'] = Business_Desc_Posiitvewords
    req_df['Business_Desc_Negativewords'] = Business_Desc_Negativewords
    req_df['Business_Desc_List_Positivevewords'] = Business_Desc_List_Positivevewords
    req_df['Business_Desc_List_Negativevewords'] = Business_Desc_List_Negativevewords
    req_df['Business_Desc_Gunning_Fog'] = Business_Desc_Gunning_Fog
    
    #######Risk Factors
    
    Risk_Factors_Posiitvewords = []
    Risk_Factors_Negativewords = []
    Risk_Factors_List_Positivevewords = []
    Risk_Factors_List_Negativevewords = []
    Risk_Factors_Gunning_Fog = []
    
    for row in req_df['Risk_Factors']:
        Risk_Factors_Posiitvewords.append(tone_count_with_negation_check(dictionary,row)[1])
        Risk_Factors_Negativewords.append(tone_count_with_negation_check(dictionary,row)[2])
        Risk_Factors_List_Positivevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[3]))))
        Risk_Factors_List_Negativevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[4]))))
        Risk_Factors_Gunning_Fog.append(textstat.gunning_fog(row))
       
    req_df['Risk_Factors_Posiitvewords'] = Risk_Factors_Posiitvewords
    req_df['Risk_Factors_Negativewords'] = Risk_Factors_Negativewords
    req_df['Risk_Factors_List_Positivevewords'] = Risk_Factors_List_Positivevewords
    req_df['Risk_Factors_List_Negativevewords'] = Risk_Factors_List_Negativevewords
    req_df['Risk_Factors_Gunning_Fog'] = Risk_Factors_Gunning_Fog
    
    ######## Management Discussion
    Management_Discussion_Posiitvewords = []
    Management_Discussion_Negativewords = []
    Management_Discussion_List_Positivevewords = []
    Management_Discussion_List_Negativevewords = []
    Management_Discussion_Gunning_Fog = []
    
    for row in req_df['Management_Discussion']:
        Management_Discussion_Posiitvewords.append(tone_count_with_negation_check(dictionary,row)[1])
        Management_Discussion_Negativewords.append(tone_count_with_negation_check(dictionary,row)[2])
        Management_Discussion_List_Positivevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[3]))))
        Management_Discussion_List_Negativevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[4]))))
        Management_Discussion_Gunning_Fog.append(textstat.gunning_fog(row))
       
    req_df['Management_Discussion_Posiitvewords'] = Management_Discussion_Posiitvewords
    req_df['Management_Discussion_Negativewords'] = Management_Discussion_Negativewords
    req_df['Management_Discussion_List_Positivevewords'] = Management_Discussion_List_Positivevewords
    req_df['Management_Discussion_List_Negativevewords'] = Management_Discussion_List_Negativevewords
    req_df['Management_Discussion_Gunning_Fog'] = Management_Discussion_Gunning_Fog
        
    
    ######Market_Risk
    
    Market_Risk_Posiitvewords = []
    Market_Risk_Negativewords = []
    Market_Risk_List_Positivevewords = []
    Market_Risk_List_Negativevewords = []
    Market_Risk_Gunning_Fog = []
    
    for row in req_df['Market_Risk']:
        Market_Risk_Posiitvewords.append(tone_count_with_negation_check(dictionary,row)[1])
        Market_Risk_Negativewords.append(tone_count_with_negation_check(dictionary,row)[2])
        Market_Risk_List_Positivevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[3]))))
        Market_Risk_List_Negativevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[4]))))
        Market_Risk_Gunning_Fog.append(textstat.gunning_fog(row))
       
    req_df['Market_Risk_Posiitvewords'] = Market_Risk_Posiitvewords
    req_df['Market_Risk_Negativewords'] = Market_Risk_Negativewords
    req_df['Market_Risk_List_Positivevewords'] = Market_Risk_List_Positivevewords
    req_df['Market_Risk_List_Negativevewords'] = Market_Risk_List_Negativevewords
    req_df['Market_Risk_Gunning_Fog'] = Market_Risk_Gunning_Fog
        
    
    ###### Financial_Statements
    
    Legal_Proceedings_Posiitvewords = []
    Legal_Proceedings_Negativewords = []
    Legal_Proceedings_List_Positivevewords = []
    Legal_Proceedings_List_Negativevewords = []
    Legal_Proceedings_Gunning_Fog = []
    
    for row in req_df['Legal_Proceedings']:
        Legal_Proceedings_Posiitvewords.append(tone_count_with_negation_check(dictionary,row)[1])
        Legal_Proceedings_Negativewords.append(tone_count_with_negation_check(dictionary,row)[2])
        Legal_Proceedings_List_Positivevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[3]))))
        Legal_Proceedings_List_Negativevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[4]))))
        Legal_Proceedings_Gunning_Fog.append(textstat.gunning_fog(row))
       
    req_df['Legal_Proceedings_Posiitvewords'] = Legal_Proceedings_Posiitvewords
    req_df['Legal_Proceedings_Negativewords'] = Legal_Proceedings_Negativewords
    req_df['Legal_Proceedings_List_Positivevewords'] = Legal_Proceedings_List_Positivevewords
    req_df['Legal_Proceedings_List_Negativevewords'] = Legal_Proceedings_List_Negativevewords
    req_df['Legal_Proceedings_Gunning_Fog'] = Legal_Proceedings_Gunning_Fog
        
        
     ###### Controls_Proc
    
    Controls_Proc_Posiitvewords = []
    Controls_Proc_Negativewords = []
    Controls_Proc_List_Positivevewords = []
    Controls_Proc_List_Negativevewords = []
    Controls_Proc_Gunning_Fog = []
    
    for row in req_df['Controls_Proc']:
        Controls_Proc_Posiitvewords.append(tone_count_with_negation_check(dictionary,row)[1])
        Controls_Proc_Negativewords.append(tone_count_with_negation_check(dictionary,row)[2])
        Controls_Proc_List_Positivevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[3]))))
        Controls_Proc_List_Negativevewords.append(",".join(map(str, remove_duplicate(tone_count_with_negation_check(dictionary,row)[4]))))
        Controls_Proc_Gunning_Fog.append(textstat.gunning_fog(row))
       
    req_df['Controls_Procs_Posiitvewords'] = Controls_Proc_Posiitvewords
    req_df['Controls_Proc_Negativewords'] = Controls_Proc_Negativewords
    req_df['Controls_Proc_List_Positivevewords'] = Controls_Proc_List_Positivevewords
    req_df['Controls_Proc_List_Negativevewords'] = Controls_Proc_List_Negativevewords
    req_df['Controls_Procs_Gunning_Fog'] = Controls_Proc_Gunning_Fog
        
    

    
    return req_df


sentiments_each_item(lmdict,parsed10Q,'Phillips 66')

,CompanyName,TypeOfFiling,Year,Quater,Business_Desc,Risk_Factors,Management_Discussion,Market_Risk,Legal_Proceedings,Controls_Proc,...,Legal_Proceedings_Posiitvewords,Legal_Proceedings_Negativewords,Legal_Proceedings_List_Positivevewords,Legal_Proceedings_List_Negativevewords,Legal_Proceedings_Gunning_Fog,Controls_Procs_Posiitvewords,Controls_Proc_Negativewords,Controls_Proc_List_Positivevewords,Controls_Proc_List_Negativevewords,Controls_Procs_Gunning_Fog
0,Phillips 66,10-Q,2018,2,Item 1. FINANCIAL STATEMENTS Consolidated Stat...,Item 1A. RISK FACTORS There have been no mat...,Item 2. MANAGEMENT'S DISCUSSION AND ANALYSI...,Item 3. QUANTITATIVE AND QUALITATIVE DISCLOS...,Item 1. LEGAL PROCEEDINGS The following is a...,Item 4. CONTROLS AND PROCEDURES We maintain ...,...,1,17,resolve,"adversely,alleged,violations,limitations,penal...",41.72,0,1,,disclosed,49.66
1,Phillips 66,10-Q,2018,3,Item 1. FINANCIAL STATEMENTS Consolidated Stat...,Item 1A. RISK FACTORS There have been no mat...,Item 2. MANAGEMENT'S DISCUSSION AND ANALYSI...,Item 3. QUANTITATIVE AND QUALITATIVE DISCLOS...,Item 1. LEGAL PROCEEDINGS The following is a...,Item 4. CONTROLS AND PROCEDURES We maintain ...,...,0,12,,"adversely,alleged,violations,limitations,penal...",33.89,0,1,,disclosed,49.66
2,Phillips 66,10-Q,2018,4,Item 1. FINANCIAL STATEMENTS Consolidated Stat...,Item 1A. RISK FACTORS There were no material...,Item 2. MANAGEMENT'S DISCUSSION AND ANALYSI...,Item 3. QUANTITATIVE AND QUALITATIVE DISCLOS...,Item 1. LEGAL PROCEEDINGS The following is a...,Item 4. CONTROLS AND PROCEDURES We maintain ...,...,2,21,resolve,"adversely,alleged,violations,limitations,penal...",28.28,0,1,,disclosed,35.73


In [24]:
#check the sentiments and gunning fog of each items in the database for the input company
#function to get the data for all the companies
def all_companies_sentiments():
    companies_list = list(parsed10Q['CompanyName'])
    #print(companies_list)
    sentiment_df = pd.DataFrame(columns= ['CompanyName','Year','Quater',
                                     'Business_Desc_Posiitvewords','Business_Desc_Negativewords','Business_Desc_List_Positivevewords','Business_Desc_List_Negativevewords','Business_Desc_Gunning_Fog',
                                    'Risk_Factors_Posiitvewords','Risk_Factors_Negativewords','Risk_Factors_List_Positivevewords','Risk_Factors_List_Negativevewords','Risk_Factors_Gunning_Fog',
                                    'Management_Discussion_Posiitvewords','Management_Discussion_Negativewords','Management_Discussion_List_Positivevewords','Management_Discussion_List_Negativevewords','Management_Discussion_Gunning_Fog',
                                    'Market_Risk_Posiitvewords','Market_Risk_Negativewords','Market_Risk_List_Positivevewords','Market_Risk_List_Negativevewords','Market_Risk_Gunning_Fog',
                                    'Legal_Proceedings_Posiitvewords','Legal_Proceedings_Negativewords','Legal_Proceedings_List_Positivevewords','Legal_Proceedings_List_Negativevewords','Legal_Proceedings_Gunning_Fog',
                                    'Controls_Proc_Posiitvewords','Controls_Proc_Negativewords','Controls_Proc_List_Positivevewords','Controls_Proc_List_Negativevewords','Controls_Proc_Gunning_Fog',
                                    'Total_Gunning_Fog'  ])
    
    for company in companies_list:
        temp_df = sentiments_each_item(lmdict,parsed10Q, company)
        sentiment_df = pd.concat([sentiment_df, temp_df], ignore_index=True)

    sentiment_df = sentiment_df[['CompanyName','Year','Quater',
                                  'Business_Desc_Posiitvewords','Business_Desc_Negativewords','Business_Desc_List_Positivevewords','Business_Desc_List_Negativevewords','Business_Desc_Gunning_Fog',
                                  'Risk_Factors_Posiitvewords','Risk_Factors_Negativewords','Risk_Factors_List_Positivevewords','Risk_Factors_List_Negativevewords','Risk_Factors_Gunning_Fog',
                                  'Management_Discussion_Posiitvewords','Management_Discussion_Negativewords','Management_Discussion_List_Positivevewords','Management_Discussion_List_Negativevewords','Management_Discussion_Gunning_Fog',
                                  'Market_Risk_Posiitvewords','Market_Risk_Negativewords','Market_Risk_List_Positivevewords','Market_Risk_List_Negativevewords','Market_Risk_Gunning_Fog',
                                  'Legal_Proceedings_Posiitvewords','Legal_Proceedings_Negativewords','Legal_Proceedings_List_Positivevewords','Legal_Proceedings_List_Negativevewords','Legal_Proceedings_Gunning_Fog',
                                  'Controls_Proc_Posiitvewords','Controls_Proc_Negativewords','Controls_Proc_List_Positivevewords','Controls_Proc_List_Negativevewords','Controls_Proc_Gunning_Fog']]
        


        
    return sentiment_df


/Users/bhumikakhandelwal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,CompanyName,Year,Quater,Business_Desc_Posiitvewords,Business_Desc_Negativewords,Business_Desc_List_Positivevewords,Business_Desc_List_Negativevewords,Business_Desc_Gunning_Fog,Risk_Factors_Posiitvewords,Risk_Factors_Negativewords,...,Legal_Proceedings_Posiitvewords,Legal_Proceedings_Negativewords,Legal_Proceedings_List_Positivevewords,Legal_Proceedings_List_Negativevewords,Legal_Proceedings_Gunning_Fog,Controls_Proc_Posiitvewords,Controls_Proc_Negativewords,Controls_Proc_List_Positivevewords,Controls_Proc_List_Negativevewords,Controls_Proc_Gunning_Fog
0,FEDEX CORP,2018,1,0,0,,,0.80,5,81,...,0,0,,,21.27,NaN,1,effective,disclosed,NaN
1,FEDEX CORP,2018,3,0,0,,,0.80,3,7,...,0,0,,,21.27,NaN,1,effective,disclosed,NaN
2,FEDEX CORP,2018,4,0,0,,,0.80,3,7,...,0,0,,,21.27,NaN,1,effective,disclosed,NaN
3,FEDEX CORP,2019,1,0,0,,,0.80,2,22,...,0,0,,,21.27,NaN,1,effective,disclosed,NaN
4,Seagate Technology plc,2018,1,53,190,"improvements,gain,leading,improve,innovation,e...","restructuring,loss,deficit,impairment,critical...",16.46,0,2,...,0,0,,,11.86,NaN,0,effective,,NaN
5,Seagate Technology plc,2018,2,59,199,"improvements,gain,leading,improve,innovation,e...","restructuring,loss,deficit,impairment,critical...",16.05,0,2,...,0,0,,,11.86,NaN,0,effective,,NaN
6,Seagate Technology plc,2018,4,0,0,,,0.80,0,0,...,0,0,,,0.80,NaN,0,,,NaN
7,Seagate Technology plc,2019,1,55,193,"improvements,gain,leading,effective,satisfies,...","restructuring,loss,deficit,critical,volatile,m...",18.81,0,2,...,0,0,,,11.86,NaN,0,effective,,NaN
8,ORACLE CORP,2018,1,61,184,"exclusive,gains,benefit,effective,favorable,im...","doubtful,loss,restructuring,losses,omitted,mis...",21.67,0,1,...,0,0,,,30.49,NaN,19,"effective,achieving,succeed","disclosed,limitations,errors,fraud,faulty,brea...",NaN
9,ORACLE CORP,2018,3,58,172,"exclusive,gains,benefit,improvements,improving...","doubtful,loss,restructuring,losses,omitted,mis...",19.50,0,1,...,0,0,,,30.49,NaN,19,"effective,adequately,achieving,succeed","disclosed,limitations,errors,fraud,faulty,brea...",NaN


# Merge S&P 500

In [25]:
os.chdir('/Users/bhumikakhandelwal/Documents/Documents/Documents/Academics/Capstone/ProjectWork/Capstone_NLP_DIWD/SEC_Script/')
datafile = pd.read_excel('S&P500 Static Data Fields.xlsx')

In [27]:
companies_name = np.unique(list(parsed10Q['CompanyName']))
companies_name = [x.lower() for x in companies_name]
datafile['NAME'] = datafile['NAME'].str.lower()
#df = datafile[datafile['NAME'].isin(companies_name)]

#print(companies_name)
df = pd.DataFrame()
data = []
for x in companies_name:
    data.append(datafile[datafile['NAME'].str.contains(x)])

df = df.append(data)
df = df.reset_index()
df.drop(df.columns[0],axis=1,inplace=True)


In [29]:

#parsed10K   #df
senti_df = all_companies_sentiments()
senti_df['CompanyName'] = senti_df['CompanyName'].str.lower()
senti_df = senti_df.reset_index(drop=True)
df = df.reset_index(drop=True)

/Users/bhumikakhandelwal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [30]:
## join two df's with left join
Finaldf = senti_df.merge(df,left_on = 'CompanyName', right_on = 'NAME' )
Finaldf.to_excel("output_dataset_10Q.xlsx")

# Insert rows 

In [16]:
# ## move the dataframe to azure database
# import pyodbc
# import sqlalchemy
# #connStr = pyodbc.connect("DRIVER={FreeTDS};SERVER= diwd.database.windows.net;DATABASE=Capstone2019;Trusted_Connection=yes")
# connStr = pyodbc.connect('DSN=MYMSSQL;UID=azureuser@diwd.database.windows.net;PWD=Capstone2019')
# crsr = connStr.cursor()

# companies_sentiments = all_companies_sentiments()

# ##delete rows from the table to avoid duplictae rows 
# crsr.execute("DELETE FROM SEC_10Q")
# for index,row in companies_sentiments.iterrows():
#     crsr.execute("INSERT INTO SEC_10Q([Company],[Years],[Quater],[Business_Desc_PositiveWords],[Business_Desc_NegativeWords],[Business_Desc_ListPositive],[Business_Desc_ListNegative],[Risk_Factors_PositiveWords],[Risk_Factors_NegativeWords],[Risk_Factors_ListPositive],[Risk_Factors_ListNegative],[Management_Disc_PositiveWords],[Management_Disc_NegativeWords],[Management_Disc_ListPositive],[Management_Disc_ListNegative],[Market_Risk_PositiveWords],[Market_Risk_NegativeWords],[Market_Risk_ListPositive],[Market_Risk_ListNegative],[Legal_PositiveWords],[Legal_NegativeWords],[Legal_ListPositive],[Legal_ListNegative],[Control_Proc_PositiveWords],[Control_Proc_NegativeWords],[Control_Proc_ListPositive],[Control_Proc_ListNegative],[Business_Desc_Fog],[Risk_Factors_Fog],[Management_Disc_Fog],[Market_Risk_Fog],[Legal_Fog],[Control_Proc_Fog],[Total_Fog]) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
#                    row['Company'],row['Year'], row['Quater'],row['Business_Desc_Posiitvewords'],row['Business_Desc_Negativewords'],row['Business_Desc_List_Positivevewords'], row['Business_Desc_List_Negativevewords'], row['Risk_Factors_Posiitvewords'],row['Risk_Factors_Negativewords'],row['Risk_Factors_List_Positivevewords'],row['Risk_Factors_List_Negativevewords'],
#                 row['Management_Discussion_Posiitvewords'],row['Management_Discussion_Negativewords'],row['Management_Discussion_List_Positivevewords'],row['Management_Discussion_List_Negativevewords'],
#                 row['Market_Risk_Posiitvewords'],row['Market_Risk_Negativewords'],row['Market_Risk_List_Positivevewords'],row['Market_Risk_List_Negativevewords'],
#                 row['Legal_Proceedings_Posiitvewords'],row['Legal_Proceedings_Negativewords'], row['Legal_Proceedings_List_Positivevewords'],row['Legal_Proceedings_List_Negativevewords'],
#                 row['Controls_Proc_Posiitvewords'],row['Controls_Proc_Negativewords'],row['Controls_Proc_List_Positivevewords'],row['Controls_Proc_List_Negativevewords'],
#                 row['Business_Desc_Gunning_Fog'],row['Risk_Factors_Gunning_Fog'],row['Management_Discussion_Gunning_Fog'],row['Market_Risk_Gunning_Fog'],row['Legal_Proceedings_Gunning_Fog'],row['Controls_Proc_Gunning_Fog'],row['Total_Gunning_Fog'])
   

#     crsr.commit()
# crsr.close()
# connStr.close()





/Users/bhumikakhandelwal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Word Cloud

In [67]:
# import matplotlib.pyplot as plt
# import wordcloud
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


# def word_cloud(company):
#     sentiment_df = sentiments_each_item(lmdict,parsed10K,company)
#     print(company)
    
#     cols = ['List_Positivevewords','List_Negativevewords']
        
#     for i in range(5):
#         for x in cols:
#             print(sentiment_df['Section'][i])
#             print(x)
#             if sentiment_df['Posiitvewords'][i] and sentiment_df['Negativewords'][i] !='0':
#                 corpus = sentiment_df[x][i]

#                 #world cloud
#                 unique_string=(" ").join(corpus)
#                 wordcloud = WordCloud(width = 1000, height = 500, background_color="white").generate(unique_string)
#                 plt.figure(figsize=(15,8))
#                 plt.imshow(wordcloud)
#                 plt.axis("off")
#                 plt.savefig("your_file_name"+".png", bbox_inches='tight')
#                 plt.show()
#                 plt.close()



In [ ]:
# def calculate_complexity():
#     for i in range(len(sentiment_df)):
#         itemText = parsed10K.loc[parsed10K.CompanyName == sentiment_df['Company'][0], sentiment_df['Section'][i]]
#         sentiment_df['ComplexityIndex'][i] = textstat.gunning_fog(itemText.values.item())
#     return sentiment_df

# sentiment_df['ComplexityIndex'] = 'NA'
# sentiment_df = calculate_complexity() 
# sentiment_df

# calculate_complexity()